In [1]:
import math
import random
import statistics
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from timeit import default_timer
from numpy import unique, ravel
from numpy import sqrt, dot, array, diagonal, mean, transpose, eye, diag, ones
from numpy import transpose, diag, dot
from numpy.linalg import svd, inv, qr, det
from sklearn.linear_model import LinearRegression

In [2]:
def topPages(__PRs, __U, __n = 10, TR=False):
    dd = {i: __PRs[i] for i in range(len(__PRs))}
    dd = {__U[page]: rank for page, rank in sorted(dd.items(), key=lambda kv: kv[1], reverse=True)}
    __pages = list(dd.keys())[:__n]
    dd= {page: dd[page] for page in __pages}
    if TR:
        return {"Link": list(dd.keys()), "Value": ravel(list(dd.values()))}
    return dd
    

def viewTopPages(__PRs, __U, __n = 10):
    if len(__PRs) == 1:
        __PRs = __PRs[0]
    __topPages = topPages(__PRs, __U, __n)
    for __page in __topPages:
        print(__page, __topPages[__page])

In [3]:
import re
import requests
from bs4 import BeautifulSoup

In [4]:
link_raw = "https://www.baseball-reference.com/leagues/majors/2021-schedule.shtml"
link_data = requests.get(link_raw, headers={"Content-Type": "html"})
soup = BeautifulSoup(link_data.text, 'html.parser')
p = soup.find_all('p')

In [5]:
# first one
p[15]

<p class="game">
<a href="/teams/CHW/2021.shtml">Chicago White Sox</a>
 (3)
 @
 <strong> <a href="/teams/LAA/2021.shtml">Los Angeles Angels</a>
 (4)</strong>
     <em><a href="/boxes/ANA/ANA202104010.shtml">Boxscore</a></em>
</p>

In [6]:
team_1 = []
score_1 = []
team_2 = []
score_2 = []

# 1,2,4,5
for i in range(15, 2686):
    game = str(p[i]).split("\n")
    if len(game) == 8:
        team1 = game[1].split("\">")[1][:-4]
        team_1.append(team1)
        team2 = game[4].split("\">")[1][:-4]
        team_2.append(team2)
        score1 = re.sub("[^0-9]","",game[2])
        score_1.append(score1)
        score2 = re.sub("[^0-9]","",game[5])
        score_2.append(score2)

score_1 = [int(s) for s in score_1]
score_2 = [int(s) for s in score_2]

In [7]:
def getOutcome(s1, s2):
    if s1 == s2:
        return 0
    if s1 > s2:
        return 1
    else:
        return 2

winning_team = [getOutcome(s1, s2) for s1, s2 in zip(score_1, score_2)]

In [8]:
df = pd.DataFrame({"Team1": team_1, "Team2": team_2, "Score1": score_1, "Score2": score_2, "WinningTeam": winning_team})

In [9]:
df.head()

,Team1,Team2,Score1,Score2,WinningTeam
0,Chicago White Sox,Los Angeles Angels,3,4,2
1,Pittsburgh Pirates,Chicago Cubs,5,3,1
2,St. Louis Cardinals,Cincinnati Reds,11,6,1
3,Los Angeles Dodgers,Colorado Rockies,5,8,2
4,Cleveland Indians,Detroit Tigers,2,3,2


In [10]:
team_names = list(unique(team_1 + team_2))

# which is also the number of teams in the MLB
print(len(team_names))

30


In [11]:
p = len(team_names)
A = np.zeros((p,p))
A = np.matrix(A)

for i in range(1000, len(df)-6):
    # if team 1 won, then team 2 feeds/adds "dominance" to team 1
    if df.iloc[i]["WinningTeam"] == 1:
        ind_i = team_names.index(df.iloc[i]["Team2"])
        ind_j = team_names.index(df.iloc[i]["Team1"])
        A[ind_i, ind_j] += 1
    # vice versa
    if df.iloc[i]["WinningTeam"] == 2:
        ind_i = team_names.index(df.iloc[i]["Team1"])
        ind_j = team_names.index(df.iloc[i]["Team2"])
        A[ind_i, ind_j] += 1

In [12]:
# p = 30 teams
# wins is just column sum
wins = [np.sum(A[:, j]) for j in range(p)]
losses = [np.sum(A[i]) for i in range(p)]
WL_ratio = [w/l for w,l in zip(wins, losses)]

In [13]:
# I've compared it to https://www.baseball-reference.com/leagues/majors/2021-standings.shtml
# its correct, and includes a couple more games
# 2021 summary
dd_teams = {"Name": team_names, "Wins": wins, "Losses": losses, "WLRatio": WL_ratio}
df_teams = pd.DataFrame(dd_teams)

In [14]:
df_teams

,Name,Wins,Losses,WLRatio
0,Arizona D'Backs,32.0,62.0,0.516129
1,Atlanta Braves,65.0,41.0,1.585366
2,Baltimore Orioles,30.0,65.0,0.461538
3,Boston Red Sox,56.0,48.0,1.166667
4,Chicago Cubs,33.0,62.0,0.532258
5,Chicago White Sox,51.0,47.0,1.085106
6,Cincinnati Reds,49.0,48.0,1.020833
7,Cleveland Indians,43.0,54.0,0.796296
8,Colorado Rockies,47.0,46.0,1.021739
9,Detroit Tigers,49.0,46.0,1.065217


In [15]:
p = len(team_names)
A = np.zeros((p,p))
A = np.matrix(A)

for i in range(len(df)-40):
    # if team 1 won, then team 2 feeds/adds "dominance" to team 1
    if df.iloc[i]["WinningTeam"] == 1:
        ind_i = team_names.index(df.iloc[i]["Team2"])
        ind_j = team_names.index(df.iloc[i]["Team1"])
        A[ind_i, ind_j] += 1
    # vice versa
    if df.iloc[i]["WinningTeam"] == 2:
        ind_i = team_names.index(df.iloc[i]["Team1"])
        ind_j = team_names.index(df.iloc[i]["Team2"])
        A[ind_i, ind_j] += 1

In [16]:
# w count - l count
def makeB(_A, p): # p is variable count
    B = np.zeros((p,p))
    B = np.matrix(B)
    for i in range(p):
        for j in range(p):
            if i != j:
                c1 = _A[i, j]
                c2 = _A[j, i]
                if c2 > c1 and c1 != 0:
                    B[j,i] = c2 - c1
                elif c1 > c2 and c2 != 0:
                    B[i,j] = c1 - c2
    return B

# 1 0 who better
def makeC(_A, p): # p is variable count
    C = np.zeros((p,p))
    C = np.matrix(C)
    for i in range(p):
        for j in range(p):
            if i != j:
                c1 = _A[i, j]
                c2 = _A[j, i]
                if c2 > c1 and c1 != 0:
                    C[j,i] = 1
                elif c1 > c2 and c2 != 0:
                    C[i,j] = 1
    return C

In [17]:

def makeD(_A, p):  # p is variable count
    D = np.zeros((p,p))
    D = np.matrix(C)
    for i in range(len(_A)):
        for j in range(len(_A[0])):
            rs = np.sum(_A[i])
            cs = np.sum(_A[:, j])
            ts = rs + cs - _A[i,j]
            D[i,j] =  _A[i,j]/ts
    return D

B = makeB(A, p)
C = makeB(A, p)
D = makeD(A, p)

In [18]:
n = len(A)
ninv = 0.5/n
TP = 0.6
TP_away = (1-TP)


class Page(object):
    def __init__(self, index):
        self.index = index
        self.in_links = []
        self.out_links = []
        self.rank = 0.01

class Network(object):
    def setupLinks(self):
        for page in self.pages:
            for i in range(len(self.A)):
                if self.A[page.index, i] > 0:
                    page.out_links.append(self.pages[i])
                    
                if self.A[i, page.index] > 0:
                    page.in_links.append(self.pages[i])
    
    def __init__(self, adj_mtx):
        self.A = adj_mtx
        self.pages = [Page(index) for index in range(len(adj_mtx))]
        self.setupLinks()

    def surf(self, max_iter):
        for itr in range(max_iter):
            for page in self.pages:
                page.rank = TP * sum([p.rank / len(p.out_links) for p in page.in_links]) + TP_away

    def getPageRanks(self):
        norm = 1 / sum([page.rank for page in self.pages])
        return [norm * page.rank for page in self.pages]
        
# clear it cuz python is silly sometimes
network = None
# and the "surf" iterations (calculates page rank) stack

# note that A vs A.T should be the best and the worst teams
network = Network(B.copy())
network.surf(100)

In [19]:
iter_ranks = network.getPageRanks()
viewTopPages(iter_ranks, team_names)

San Francisco Giants 0.051591353907535706
Houston Astros 0.043446585610453406
Los Angeles Dodgers 0.04306777600202822
New York Yankees 0.04199073615897966
Milwaukee Brewers 0.04113300827885344
Chicago White Sox 0.04048241607728829
Seattle Mariners 0.03977366769608425
Tampa Bay Rays 0.03957816271744414
St. Louis Cardinals 0.03855696690884009
Detroit Tigers 0.03840336426958092


In [20]:
n = len(A)

In [21]:
def convertToP(_A, _p=0.85):
    tp = 0.85
    p1 = (1-_p)/n
    p2 = 1.0/n
    P = []
    for i in range(n):
        r = []
        ri = np.sum(_A[i])
        if np.sum(_A[i]) > 0:
            for j in range(n):
                r.append(_p*(_A[i,j]/ri) + p1)
        else:
            for j in range(n):
                r.append(p2)
        P.append(r)
    return array(P)

def topTeamsFromMtx(__A, tp=0.85):
    P = convertToP(__A, tp)
    IP = np.eye(n) - P.T
    b = np.zeros(n).reshape(n,1)
    IP[0] = ones(n)
    b[0] = 1.0
    IP[0] = ones(n)
    u,d,v = svd(IP)
    x = dot(v.T, inv(diag(d))).dot(u.T).dot(b)
    viewTopPages(x, team_names)

In [22]:
topTeamsFromMtx(A,2)

San Francisco Giants [0.06294426]
Los Angeles Dodgers [0.062425]
San Diego Padres [0.04854906]
Milwaukee Brewers [0.04586482]
Houston Astros [0.04501517]
St. Louis Cardinals [0.04411282]
Seattle Mariners [0.04115066]
Colorado Rockies [0.03873421]
Cincinnati Reds [0.03723449]
Oakland Athletics [0.0368022]


In [23]:
P = convertToP(C.copy())

n = len(B)
tp = 0.85
ri_not0 = (1-tp)/float(n)
IP = np.eye(n) - P.T
np.sum(IP[:,0])
IP[0] = ones(n)
b = np.zeros(n).reshape(n,1)
b[0] = 1.0

# The final Rankings Using Google's Pagerank Algorithm  

In [24]:
u,d,v = svd(IP)
x = dot(v.T, inv(diag(d))).dot(u.T).dot(b)
viewTopPages(x, team_names)

San Francisco Giants [0.06137736]
Houston Astros [0.0600877]
Seattle Mariners [0.05902641]
Detroit Tigers [0.05356551]
New York Yankees [0.05237637]
Tampa Bay Rays [0.05209038]
Los Angeles Dodgers [0.04765206]
St. Louis Cardinals [0.04667843]
Milwaukee Brewers [0.04147153]
Atlanta Braves [0.03755214]
